In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import requests
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")



c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--Salesforce--blip-image-captioning-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [8]:
img_url = 'C:\\Users\\Admin\\Downloads\\resturent.jpg' 
# raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')
raw_image = Image.open(img_url).convert('RGB')
# conditional image captioning
text = input('Enter the text: ').lower()
inputs = processor(raw_image, text, return_tensors="pt")
out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))
# >>> a photography of a woman and her dog


what is this image of a restaurant?


In [10]:
print(out)

tensor([[30522,  2054,  2003,  2023,  3746,  1997,  1037,  4825,  1029,   102]])


In [ ]:
# The Architecture if txt input -> functin 1 if only image function 1 else if input contains both fucntion 2 it makes easliy understandability and durability of code ( if  Objects are created its beter)


In [25]:
from transformers import pipeline
from openai import OpenAI
import base64
from dotenv import load_dotenv
# pipe = pipeline(
#     'image-to-text',
#     model="Salesforce/blip-image-captioning-base",
#     device='cpu',
#     torch_dtype='torch.bfloat16' if torch.cuda.is_available() else 'float32'

# )
load_dotenv()
client = OpenAI()
def image_to_base64(image_path:str)-> str:
    '''
    Function decode the image
    Args: Image_path
    Returns:
      Base64 decode image
    '''
    with open(image_path, 'rb') as img:
        image = img.read().decode('utf-8')
        decoded_imge = base64.b64encode(image)
        # return base64.b64encode(img.read()).decode('utf-8')
        return decoded_imge

def run_agent(image_src: str = None, text_input: str= None)-> str:
    '''
    Input: 
    The fucntion that takes image and text as input 
    Return:
        Model Generated Text
    '''
    content = []
    if text_input:
        content.append({'type':'text', 'text': text_input})
    if image_src:
        decoded_image = image_to_base64(image_path=image_src)
        content.append({
            'type': 'image_url',
            'image_url':{
                "url": f"data:image/jpeg;base64,{decoded_image}"

            }
        })
    
    response= client.chat.completions.create(
        model = "gpt-4.1",
        messages=[
            {"role": "user", "content": content}
        ],
        max_tokens=300
    )
    return response.choices[0].message.content

In [26]:
# Text only
print(run_agent(text_input="Explain the concept of entropy in physics."))

# Image only
# print(run_agent(image_path="sample.jpg"))

# Text + Image
# print(run_agent(text_input="What is happening in this image?", image_path="sample.jpg"))


**Entropy** in physics is a measure of the disorder, randomness, or uncertainty within a physical system. It is a central concept in thermodynamics and statistical mechanics.

### Key Points

- **Thermodynamics:**  
  Entropy quantifies the amount of energy in a system that is unavailable for doing work. When energy is transformed (like heat into work), some energy becomes less useful, and entropy increases. According to the *Second Law of Thermodynamics*, the total entropy of an isolated system can never decrease; it tends to increase over time, leading to irreversibility in natural processes.

- **Statistical Interpretation:**  
  From a microscopic perspective (statistical mechanics), entropy measures the number of possible microscopic arrangements (microstates) that correspond to the same macroscopic state. More microstates mean higher entropy. The famous formula is:
  \[
  S = k_B \ln W
  \]
  where \( S \) is entropy, \( k_B \) is Boltzmann's constant, and \( W \) is the number o